# LangChain QA Panel App

This notebook shows how to make this app. 
STILL SOME ISSUES

### Source
Source: 
- Sophia Yang on youtube: https://www.youtube.com/watch?v=IvEh7A308FU
- Huggingface space Sophia: https://sophiamyang-panel-pdf-qa.hf.space/LangChain_QA_Panel_App

In [16]:
!pip install langchain openai chromadb tiktoken pypdf panel

  Using cached openai-0.27.4-py3-none-any.whl (70 kB)
  Using cached chromadb-0.3.21-py3-none-any.whl (46 kB)
  Using cached tiktoken-0.3.3-cp310-cp310-macosx_10_9_x86_64.whl (735 kB)
  Using cached pypdf-3.7.1-py3-none-any.whl (248 kB)
  Using cached fastapi-0.95.0-py3-none-any.whl (57 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached posthog-2.4.2-py2.py3-none-any.whl (36 kB)
  Using cached clickhouse_connect-0.5.20-cp310-cp310-macosx_10_9_x86_64.whl (234 kB)
  Using cached uvicorn-0.21.1-py3-none-any.whl (57 kB)
  Using cached hnswlib-0.7.0.tar.gz (33 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached duckdb-0.7.1-cp310-cp310-macosx_10_9_x86_64.whl (14.6 MB)
  Using cached starlette-0.26.1-py3-none-any.whl (66 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached sentencepiece

  Using cached torchvision-0.15.1-cp310-cp310-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached websockets-11.0.1-cp310-cp310-macosx_10_9_x86_64.whl (120 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached watchfiles-0.19.0-cp37-abi3-macosx_10_7_x86_64.whl (405 kB)
  Using cached uvloop-0.17.0-cp310-cp310-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached httptools-0.5.0-cp310-cp310-macosx_10_9_x86_64.whl (159 kB)
  Using cached torch-2.0.0-cp310-none-macosx_10_9_x86_64.whl (139.8 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      running bdist_wheel
      running build
      running build_ext
      creating var
      creating var/folders
      creating var/folders/cj
      creating var/folders/cj/qtbz9fvd3svc0x28yv2756mh0000gn
      creating var/folders/cj/qtbz9fvd3svc0x28yv2756mh0000gn/T
      clan

In [ ]:
#!conda install -c conda-forge hnswlib

In [27]:
!pip install chromadb

  Using cached chromadb-0.3.21-py3-none-any.whl (46 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached clickhouse_connect-0.5.20-cp310-cp310-macosx_10_9_x86_64.whl (234 kB)
  Using cached posthog-2.4.2-py2.py3-none-any.whl (36 kB)
  Using cached duckdb-0.7.1-cp310-cp310-macosx_10_9_x86_64.whl (14.6 MB)
  Using cached uvicorn-0.21.1-py3-none-any.whl (57 kB)
  Using cached fastapi-0.95.0-py3-none-any.whl (57 kB)
  Using cached starlette-0.26.1-py3-none-any.whl (66 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Using cached sentencepiece-0.1.97-cp310-cp310-macosx_10_9_x86_64.whl (1.2 MB)
  Using cached torchvision-0.15.1-cp310-cp310-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached websockets-11.0.1-cp310-cp310-macosx_10_9_x86_64.whl (120 kB)
  Using cached uvloop-0.17.0-cp310-cp310-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached watchfiles-0.1

  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [28]:
!pip install tiktoken pypdf panel

  Using cached tiktoken-0.3.3-cp310-cp310-macosx_10_9_x86_64.whl (735 kB)
  Using cached pypdf-3.7.1-py3-none-any.whl (248 kB)


In [29]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


In [30]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

In [31]:
file_input = pn.widgets.FileInput(width=300)

openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Enter your OpenAI API Key here...", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type', 
    options=['stuff', 'map_reduce', "refine", "map_rerank"]
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings", margin=10
    ), width=600
)

In [32]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [38]:
result = qa("example.pdf", "what is the total number of AI publications?")

TypeError: qa() missing 2 required positional arguments: 'chain_type' and 'k'

In [47]:
convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value
    
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("temp.pdf")
    
        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
            #return convos
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [49]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Could not import openai python package. Please it install it with `pip install openai`. (type=value_error)

In [42]:
pwd

'/Users/michielbontenbal/Documents/GitHub/chat_with_documents'

In [48]:
!chmod -R 777 /Users/michielbontenbal/Documents/GitHub/chat_with_documents/temp.pdf

chmod: /Users/michielbontenbal/Documents/GitHub/chat_with_documents/temp.pdf: No such file or directory


In [37]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=630, scroll=True)

In [36]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F60A Question Answering with your PDF file
    
    1) Upload a PDF. 2) Enter OpenAI API key. This costs $. Set up billing at [OpenAI](https://platform.openai.com/account). 3) Type a question and click "Run".
    
    """),
    pn.Row(file_input,openaikey),
    output,
    widgets

).servable()

Traceback (most recent call last):
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/pyviz_comms/__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/protocol/messages/patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/util/callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/bokeh/util/callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/io/state.py", line 408, in _handle_exception
 raise exception
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/widgets/button.py", line 183, in _process_event
 self.param.trigger('value')
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/param/parameterized.py", line 1993, in trigger
 self_.set_param(**dict(params, **triggers))
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/param/parameterized.py", line 1929, in set_param
 return self_.update(kwargs)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/param/parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/param/parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/panel/param.py", line 838, in _replace_pane
 new_object = self.eval(self.object)
 File "/Users/michielbontenbal/anaconda3/lib/pyt

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] PasswordInput(placeholder='Enter your OpenAI A..., width=300)
    [2] WidgetBox(scroll=True, width=630)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Column, loading_indicator=True, sizing_mode='stretch_width')
    [3] Row(width=600)
        [0] Column(margin=5, sizing_mode='stretch_width')
            [0] TextEditor(height=160, placeholder='Enter your questions h..., sizing_mode='stretch_width', toolbar=False)
            [1] Button(name='Run!', sizing_mode='stretch_width')
        [1] Card(margin=10, sizing_mode='stretch_width', title='Advanced settings')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Chain type', options=['stuff', 'map_reduce', ...], sizing_mode='stretch_width', value='stuff')
                [1] IntSlider(end=5, name='Number of relevant c..., sizing_mode='stretch_width', start=1, value=2)